In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# HTML
from IPython.display import HTML
from IPython.display import display

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
plotly.offline.init_notebook_mode (connected = True)

%matplotlib inline
sns.set_style("whitegrid")

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ShuffleSplit, RepeatedKFold, train_test_split, GridSearchCV,  cross_val_score

# warning
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load train and Test set
train = pd.read_csv("../input/titanic/train.csv")
#test = pd.read_csv("../input/titanic/test.csv")


In [ ]:
import plotly.graph_objects as go

headerColor = 'royalblue'
rowEvenColor = 'paleturquoise'
rowOddColor = 'paleturquoise'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Variable</b>','<b>Definition</b>','<b>Key</b>','<b>Independent/Dependent Variable</b>','<b>Data Type </b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['center','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ["PassengerId","Name","Pclass","Sex","Age","Sibsp","Parch","Ticket","Fare","Cabin","Embarked","Survival"],
      ["PassengerId","PassengerName","Ticket class","Sex","Age in years","# of siblings / spouses aboard the Titanic","# of parents / children aboard the Titanic","Ticket number","Passenger fare","Cabin number","Port of Embarkation","Survival"],
      ["", "1 = 1st = Upper, 2 = 2nd = Middle, 3 = 3rd = Lower","",""," (Sibling =(brother, sister, stepbrother, stepsister))","(pouse = husband, wife (mistresses and fiancés were ignored))","(Parent = mother, father),(Child = daughter, son, stepdaughter, stepson),(Some children travelled only with a nanny, therefore parch=0 for them.)","","","C = Cherbourg, Q = Queenstown, S = Southampton","","0 = No, 1 = Yes"],
      ["Independent","Independent","Independent","Independent","Independent","Independent","Independent","Independent","Input/Independent","Independent","Independent","Dependent"],
      ["Numerical","Text","Ordinal","Categorical","Numerical(Continous)","Numerical(Discrete)","Numerical(Discrete)","Mixed(numeric and alphanumeric)","Numerical(Continous)","Mixed(numeric and alphanumeric)","Categorical","Categorical"]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [['white','white','white', 'white','white']*5],
    align = ['center', 'center'],
    font = dict(color = 'darkslategray', size = 15)
    ))
])

fig.show()


In [ ]:
# Outlier detection 
from collections import Counter
#Once initialized, counters are accessed just like dictionaries.
#Also, it does not raise the KeyValue error (if key is not present) instead the value’s count is shown as 0.
def detect_outliers(df,n,features):
    outlier_indices = []
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col],25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        # outlier step
        outlier_step = 1.5 * IQR
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index       
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    return multiple_outliers   
# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])
train.loc[Outliers_to_drop] # Show the outliers rows